In [1]:
!pip install numpy pillow opencv-python

In [2]:
import numpy as np
from PIL import Image
import cv2
from tqdm import tqdm

In [5]:

def load_image(image_path):
    """
    Load an image from a file.
    :param image_path: Path to the image file.
    :return: PIL Image object.
    """
    return Image.open(image_path)

def extract_color_grading(image):
    """
    Extract color grading information from the image.
    :param image: PIL Image object.
    :return: 3D numpy array representing the color grading LUT.
    """
    print("Extracting color grading LUT...")
    image_array = np.array(image).astype(np.float32) / 255.0
    lut_size = 17
    lut = np.zeros((lut_size, lut_size, lut_size, 3), dtype=np.float32)
    
    for r in tqdm(range(lut_size), desc="Processing ColorGrading LUT", unit="level"):
        for g in range(lut_size):
            for b in range(lut_size):
                # Map the LUT indices to the image range
                r_val = r / (lut_size - 1)
                g_val = g / (lut_size - 1)
                b_val = b / (lut_size - 1)
                
                # Find the closest color in the image
                # ...existing code...
                distances = np.linalg.norm(image_array[:, :, :3] - [r_val, g_val, b_val], axis=-1)
                # ...existing code...
                closest_index = np.unravel_index(np.argmin(distances), image_array.shape[:2])
                
                # Use the closest color as the LUT value
                lut[r, g, b] = image_array[closest_index][:3]
    
    return lut

def extract_contrast(image):
    """
    Extract contrast information from the image.
    :param image: PIL Image object.
    :return: 3D numpy array representing the Contrast LUT.
    """
    print("Extracting contrast LUT...")
    image_array = np.array(image).astype(np.float32) / 255.0
    lut_size = 17
    lut = np.zeros((lut_size, lut_size, lut_size, 3), dtype=np.float32)

    for r in tqdm(range(lut_size), desc="Processing Contrast LUT", unit="level"):
        for g in range(lut_size):
            for b in range(lut_size):
                # Map the LUT indices to the image range
                r_val = r / (lut_size - 1)
                g_val = g / (lut_size - 1)
                b_val = b / (lut_size - 1)
                
                # Calculate the contrast-adjusted value
                contrast_factor = 1.2  # Adjust this factor as needed
                r_val_contrast = (r_val - 0.5) * contrast_factor + 0.5
                g_val_contrast = (g_val - 0.5) * contrast_factor + 0.5
                b_val_contrast = (b_val - 0.5) * contrast_factor + 0.5
                
                # Ensure values are within [0, 1]
                r_val_contrast = max(0, min(1, r_val_contrast))
                g_val_contrast = max(0, min(1, g_val_contrast))
                b_val_contrast = max(0, min(1, b_val_contrast))
                
                lut[r, g, b] = [r_val_contrast, g_val_contrast, b_val_contrast]
    
    return lut

def extract_brightness(image):
    """
    Extract brightness information from the image.
    :param image: PIL Image object.
    :return: 3D numpy array representing the brightness LUT.
    """
    print("Extracting brightness information...")
    image_array = np.array(image).astype(np.float32) / 255.0
    lut_size = 17
    lut = np.zeros((lut_size, lut_size, lut_size, 3), dtype=np.float32)
    for r in tqdm(range(lut_size), desc="Processing Brightness LUT", unit="level"):
        for g in range(lut_size):
            for b in range(lut_size):
                # Map the LUT indices to the image range
                r_val = r / (lut_size - 1)
                g_val = g / (lut_size - 1)
                b_val = b / (lut_size - 1)
                
                # Calculate the brightness-adjusted value
                brightness_factor = 1.1  # Adjust this factor as needed
                r_val_brightness = r_val * brightness_factor
                g_val_brightness = g_val * brightness_factor
                b_val_brightness = b_val * brightness_factor
                
                # Ensure values are within [0, 1]
                r_val_brightness = max(0, min(1, r_val_brightness))
                g_val_brightness = max(0, min(1, g_val_brightness))
                b_val_brightness = max(0, min(1, b_val_brightness))
                
                lut[r, g, b] = [r_val_brightness, g_val_brightness, b_val_brightness]
    
    return lut

def apply_lut(image, lut):
    """
    Apply the LUT to an image.
    :param image: PIL Image object.
    :param lut: 3D numpy array representing the LUT.
    :return: PIL Image object with the LUT applied.
    """
    print("Applying LUT...")
    image_array = np.array(image).astype(np.float32) / 255.0
    lut_size = lut.shape[0] - 1
    indices = (image_array * lut_size).astype(np.int32)
    transformed_image = lut[indices[:, :, 0], indices[:, :, 1], indices[:, :, 2]]
    transformed_image = (transformed_image * 255).astype(np.uint8)
    return Image.fromarray(transformed_image)

def save_lut(lut, lut_path):
    """
    Save a LUT to a file.
    :param lut: 3D numpy array representing the LUT.
    :param lut_path: Path to save the LUT file.
    """
    np.save(lut_path, lut)

def load_lut(lut_path):
    """
    Load a LUT from a file.
    :param lut_path: Path to the LUT file.
    :return: 3D numpy array representing the LUT.
    """
    return np.load(lut_path)

# Example usage
if __name__ == "__main__":
    # Load the reference image
    import time
    import os

    start_time = time.time()
    run = round(start_time)

    output_dir = f"data/output/{run}"
    os.makedirs(output_dir, exist_ok=True)

    reference_image_path = 'data/images/reference_image2.jpg'
    reference_image = load_image(reference_image_path)
    print("Reference image loaded.")
    # Extract color grading, contrast, and brightness LUTs
    color_grading_lut = extract_color_grading(reference_image)
    # contrast_lut = extract_contrast(reference_image)
    # brightness_lut = extract_brightness(reference_image)
    print("LUTs extracted.")

    end_time = time.time()
    print(f" Lut processing: Execution time: {end_time - start_time} seconds")
    # Save the LUTs to files
    save_lut(color_grading_lut, f"{output_dir}/color_grading_lut.npy")
    # save_lut(contrast_lut, f"{output_dir}/contrast_lut.npy")
    # save_lut(brightness_lut, f"{output_dir}/brightness_lut.npy")
    # Load an image to apply the LUTs
    input_image_path = 'data/images/input_image.jpg'
    input_image = load_image(input_image_path)
    
    # Apply the LUTs to the input image
    color_grading_image = apply_lut(input_image, color_grading_lut)
    # contrast_image = apply_lut(input_image, contrast_lut)
    # brightness_image = apply_lut(input_image, brightness_lut)
    
    # Save the transformed images
    color_grading_image.save(f"{output_dir}/color_grading_output.jpg")
    # contrast_image.save(f"{output_dir}/contrast_output.jpg")
    # brightness_image.save(f"{output_dir}/brightness_output.jpg")
    
    end_time = time.time()
    print("Transformed images saved.")
    print(f"Execution time: {end_time - start_time} seconds")

# Explanation


# 	1.	load_image: Loads an image from a file using PIL.
# 	2.	extract_color_grading: Extracts color grading information by mapping each LUT index to the closest color in the reference image.
# 	3.	extract_contrast: Extracts contrast information by adjusting the color values based on a contrast factor.
# 	4.	extract_brightness: Extracts brightness information by adjusting the color values based on a brightness factor.
# 	5.	apply_lut: Applies the LUT to an image.
# 	6.	save_lut and load_lut: Save and load LUTs to/from files.


# Example Usage


# 	1.	Load the Reference Image: The reference image is used to create the LUTs.
# 	2.	Extract LUTs: Extract color grading, contrast, and brightness LUTs from the reference image.
# 	3.	Save LUTs: Save the LUTs to files for later use.
# 	4.	Load Input Image: Load an input image to apply the LUTs.
# 	5.	Apply LUTs: Apply the LUTs to the input image and save the transformed images.

# You can adjust the contrast and brightness factors in the `extract_contrast` and `extract_brightness` functions to achieve the desired effects.

Reference image loaded.
Extracting color grading LUT...


Processing ColorGrading LUT:   0%|          | 0/17 [00:00<?, ?level/s]

Processing ColorGrading LUT: 100%|██████████| 17/17 [03:03<00:00, 10.82s/level]

LUTs extracted.
 Lut processing: Execution time: 183.9451208114624 seconds
Applying LUT...
Transformed images saved.
Execution time: 184.06524920463562 seconds


In [ ]:
import numpy as np

def save_lut_as_cube(lut, file_path):
    """
    Save a 3D LUT as a .cube file.
    :param lut: 3D numpy array representing the LUT.
    :param file_path: Path to save the .cube file.
    """
    lut_size = lut.shape[0]
    with open(file_path, 'w') as f:
        f.write("TITLE \"Generated by LUT Convertor\"\n")
        f.write(f"LUT_3D_SIZE {lut_size}\n")
        f.write("DOMAIN_MIN 0.0 0.0 0.0\n")
        f.write("DOMAIN_MAX 1.0 1.0 1.0\n")
        
        for r in range(lut_size):
            for g in range(lut_size):
                for b in range(lut_size):
                    f.write(f"{lut[r, g, b, 0]} {lut[r, g, b, 1]} {lut[r, g, b, 2]}\n")

# Example usage
if __name__ == "__main__":
    # Load the LUT from a .npy file
    lut = np.load('data/output/color_grading_lut.npy')
    
    # Save the LUT as a .cube file
    save_lut_as_cube(lut, 'data/output/color_grading_lut.cube')

---

# LUT Convertor

LUT Convertor is a Python tool for extracting and applying Look-Up Tables (LUTs) for color grading, contrast, and brightness adjustments on images. This tool leverages the power of numpy and PIL to process images and transform their appearance based on reference images.

## Features

- Extract color grading LUT from a reference image.
- Extract contrast and brightness LUTs.
- Apply LUTs to input images.
- Save transformed images.

## Installation

1. Clone the repository:
    ```sh
    git clone https://github.com/yourusername/lut-convertor.git
    cd lut-convertor
    ```

2. Install the required dependencies:
    ```sh
    pip install -r requirements.txt
    ```

## Usage

1. Place your reference image and input image in the 

images

 directory.

2. Run the script:
    ```sh
    python lut_convertor.py
    ```

3. The transformed images will be saved in the 

output

 directory.

## Example



In [ ]:
from PIL import Image
import numpy as np
from tqdm import tqdm

def load_image(image_path):
    return Image.open(image_path)

def extract_color_grading(image):
    # Function implementation
    pass

def extract_contrast(image):
    # Function implementation
    pass

def extract_brightness(image):
    # Function implementation
    pass

def apply_lut(image, lut):
    # Function implementation
    pass

if __name__ == "__main__":
    import time

    start_time = time.time()

    reference_image_path = 'data/images/reference_image.jpg'
    input_image_path = 'data/images/input_image.jpg'
    
    reference_image = load_image(reference_image_path)
    color_grading_lut = extract_color_grading(reference_image)
    contrast_lut = extract_contrast(reference_image)
    brightness_lut = extract_brightness(reference_image)

    input_image = load_image(input_image_path)
    color_grading_image = apply_lut(input_image, color_grading_lut)
    contrast_image = apply_lut(input_image, contrast_lut)
    brightness_image = apply_lut(input_image, brightness_lut)

    color_grading_image.save('data/output/color_grading_output.jpg')
    contrast_image.save('data/output/contrast_output.jpg')
    brightness_image.save('data/output/brightness_output.jpg')

    end_time = time.time()
    print(f"Transformed images saved. Execution time: {end_time - start_time} seconds")



## License

This project is licensed under the MIT License.

---

Feel free to customize this README further to better suit your needs.